# User Activity Forecasting

Predict how many days a user will be active in the next 7 days (0-7).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import json
import random
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')

SEED = 42
np.random.seed(SEED)
random.seed(SEED)

from sklearn.model_selection import cross_val_score, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.feature_selection import VarianceThreshold

import optuna
from optuna.samplers import TPESampler
optuna.logging.set_verbosity(optuna.logging.WARNING)

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

import shap

from snowflake.snowpark.context import get_active_session
from snowflake.ml.registry import Registry
from snowflake.ml.experiment import ExperimentTracking
from snowflake.ml.feature_store import FeatureStore

session = get_active_session()
registry = Registry(session=session, database_name="dev", schema_name="data_science")
exp = ExperimentTracking(session=session)
exp.set_experiment("user_activity_forecasting")

fs = FeatureStore(
    session=session, 
    database="dev", 
    name="user_activity_feature_store", 
    default_warehouse="analytics_wh_flex"
)

In [ ]:
CONFIG = {
    "table": "DEV.DATA_SCIENCE.TRAINING_SPINE_ENGINEERED",
    "max_date": "2025-11-19",
    "lookback_days": 120,
    "target_window": 7,
    "samples_per_class_per_day": 2500,
    "train_ratio": 0.8,
    "optuna_trials": 30,
    "cv_splits": 3,
    "variance_threshold": 0.01,
    "correlation_threshold": 0.90,
    "importance_threshold": 0.01,
    "acceptable_rmse": 2.0,
    "max_overfit_gap": 0.3,
}

## 1. Load Data

In [ ]:
feature_views = [
    fs.get_feature_view(name="user_features_sessions", version="1.2"),
    fs.get_feature_view(name="user_features_ads", version="1.2"),
    fs.get_feature_view(name="user_features_call_quality", version="1.2"),
    fs.get_feature_view(name="user_features_call_rating", version="1.2"),
    fs.get_feature_view(name="user_features_data_usage", version="1.2"),
    fs.get_feature_view(name="user_features_drawer_event", version="1.2"),
    fs.get_feature_view(name="user_features_inbound_calls", version="1.2"),
    fs.get_feature_view(name="user_features_messages", version="1.2"),
    fs.get_feature_view(name="user_features_nps_rating", version="1.2"),
    fs.get_feature_view(name="user_features_outbound_calls", version="1.2"),
    fs.get_feature_view(name="user_features_redial", version="1.2"),
]
print(f"Feature views loaded: {len(feature_views)}")

In [ ]:
query = f"""
WITH sampled AS (
    SELECT *, 
           ROW_NUMBER() OVER (
               PARTITION BY date_utc, active_days_in_week 
               ORDER BY RANDOM()
           ) AS rn
    FROM {CONFIG['table']}
    WHERE date_utc >= DATEADD('day', -{CONFIG['lookback_days']}, '{CONFIG['max_date']}')
      AND date_utc <= DATEADD('day', -{CONFIG['target_window']}, '{CONFIG['max_date']}')
)
SELECT * EXCLUDE (rn) 
FROM sampled 
WHERE rn <= {CONFIG['samples_per_class_per_day']}
ORDER BY date_utc, user_id_hex
"""

spine_df = session.sql(query)
print(f"Spine rows: {spine_df.count():,}")

df = fs.generate_training_set(
    spine_df=spine_df, 
    features=feature_views, 
    spine_label_cols=["ACTIVE_DAYS_IN_WEEK"]
)
df = df.to_pandas()
df = df.sort_values(['DATE_UTC', 'USER_ID_HEX']).reset_index(drop=True)

print(f"Total rows: {len(df):,}")
print(f"Total columns: {len(df.columns)}")
print(f"Dates: {df['DATE_UTC'].nunique()} ({df['DATE_UTC'].min()} to {df['DATE_UTC'].max()})")

## 2. Validation

In [ ]:
target = df['ACTIVE_DAYS_IN_WEEK']

assert target.between(0, 7).all(), f"Target out of range: {target.min()} to {target.max()}"
assert df['USER_ID_HEX'].notna().all(), "Missing user IDs"
assert df['DATE_UTC'].notna().all(), "Missing dates"

null_pct = df.isnull().sum() / len(df) * 100
high_null = null_pct[null_pct > 20]
if len(high_null):
    print(f"Columns with >20% nulls: {list(high_null.index)}")

df = df.fillna(0)

print(f"Target distribution:")
print(target.value_counts().sort_index().to_string())

## 3. EDA

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

target.hist(bins=8, ax=axes[0], edgecolor='black')
axes[0].set_xlabel('Active Days')
axes[0].set_title('Target')

df.groupby('DATE_UTC').size().plot(ax=axes[1])
axes[1].set_title('Samples/Day')

df.groupby('DATE_UTC')['ACTIVE_DAYS_IN_WEEK'].mean().plot(ax=axes[2])
axes[2].set_title('Avg Target/Day')

plt.tight_layout()
plt.show()

In [ ]:
numeric = df.select_dtypes(include=[np.number]).columns
corr = df[numeric].corr()['ACTIVE_DAYS_IN_WEEK'].abs().sort_values(ascending=False)

print("Top correlations with target:")
for f, v in corr.iloc[1:11].items():
    print(f"  {f}: {v:.3f}")

## 4. Feature Prep

In [ ]:
exclude = ['USER_ID_HEX', 'DATE_UTC', 'ACTIVE_DAYS_IN_WEEK']
feature_cols = [c for c in df.columns if c not in exclude]

X = df[feature_cols].copy()
y = df['ACTIVE_DAYS_IN_WEEK'].copy()
dates = df['DATE_UTC'].copy()

selector = VarianceThreshold(threshold=CONFIG["variance_threshold"])
selector.fit(X)
features = X.columns[selector.get_support()].tolist()
X = X[features]

print(f"Features: {len(feature_cols)} -> {len(features)}")

## 5. Train/Test Split

In [ ]:
unique_dates = sorted(dates.unique())
n_train = int(len(unique_dates) * CONFIG['train_ratio'])
cutoff = unique_dates[n_train - 1]

train_idx = dates <= cutoff
test_idx = dates > cutoff

X_train, X_test = X[train_idx].reset_index(drop=True), X[test_idx].reset_index(drop=True)
y_train, y_test = y[train_idx].reset_index(drop=True), y[test_idx].reset_index(drop=True)
dates_train, dates_test = dates[train_idx].reset_index(drop=True), dates[test_idx].reset_index(drop=True)

assert dates_train.max() < dates_test.min(), "Data leakage: train/test dates overlap"

print(f"Train: {len(X_train):,} rows, {n_train} days ({unique_dates[0]} to {cutoff})")
print(f"Test:  {len(X_test):,} rows, {len(unique_dates)-n_train} days ({unique_dates[n_train]} to {unique_dates[-1]})")

## 6. Baselines

In [ ]:
mean_pred = np.full(len(y_test), y_train.mean())
mean_rmse = np.sqrt(mean_squared_error(y_test, mean_pred))

naive_col = 'ACTIVE_DAYS_PREV_7'
if naive_col in X_test.columns:
    naive_rmse = np.sqrt(mean_squared_error(y_test, X_test[naive_col]))
else:
    naive_col = [c for c in X_test.columns if 'PREV_7' in c.upper()]
    if naive_col:
        naive_rmse = np.sqrt(mean_squared_error(y_test, X_test[naive_col[0]]))
    else:
        naive_rmse = mean_rmse

baseline = min(mean_rmse, naive_rmse)

print(f"Mean baseline: {mean_rmse:.4f}")
print(f"Naive baseline: {naive_rmse:.4f}")
print(f"Target to beat: {baseline:.4f}")

## 7. Hyperparameter Tuning

In [ ]:
tscv = TimeSeriesSplit(n_splits=CONFIG['cv_splits'])

def objective_lgbm(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
        'subsample': trial.suggest_float('subsample', 0.7, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-6, 1.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-6, 1.0, log=True),
        'random_state': SEED, 'verbosity': -1, 'n_jobs': -1
    }
    scores = cross_val_score(LGBMRegressor(**params), X_train, y_train, 
                             cv=tscv, scoring='neg_root_mean_squared_error')
    return -scores.mean()

def objective_xgb(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.7, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),
        'gamma': trial.suggest_float('gamma', 1e-6, 0.5, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-6, 1.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-6, 1.0, log=True),
        'random_state': SEED, 'verbosity': 0, 'n_jobs': -1
    }
    scores = cross_val_score(XGBRegressor(**params), X_train, y_train, 
                             cv=tscv, scoring='neg_root_mean_squared_error')
    return -scores.mean()

In [ ]:
n_trials = CONFIG["optuna_trials"]

print("Tuning LightGBM...")
study_lgbm = optuna.create_study(direction='minimize', sampler=TPESampler(seed=SEED))
study_lgbm.optimize(objective_lgbm, n_trials=n_trials, show_progress_bar=True)

print("Tuning XGBoost...")
study_xgb = optuna.create_study(direction='minimize', sampler=TPESampler(seed=SEED))
study_xgb.optimize(objective_xgb, n_trials=n_trials, show_progress_bar=True)

results = pd.DataFrame({
    'model': ['LightGBM', 'XGBoost'],
    'cv_rmse': [study_lgbm.best_value, study_xgb.best_value]
}).sort_values('cv_rmse')

print(f"\n{results.to_string(index=False)}")
best_name = results.iloc[0]['model']

## 8. Train Model

In [ ]:
studies = {'LightGBM': (study_lgbm, LGBMRegressor), 'XGBoost': (study_xgb, XGBRegressor)}
study, ModelClass = studies[best_name]
params = study.best_params.copy()
params['random_state'] = SEED
params['n_jobs'] = -1
if best_name == 'LightGBM':
    params['verbosity'] = -1
else:
    params['verbosity'] = 0

t0 = time.time()
model = ModelClass(**params)
model.fit(X_train, y_train)
train_time = time.time() - t0

pred_train = model.predict(X_train)
pred_test = model.predict(X_test)

train_rmse = np.sqrt(mean_squared_error(y_train, pred_train))
test_rmse = np.sqrt(mean_squared_error(y_test, pred_test))
test_mae = mean_absolute_error(y_test, pred_test)
test_r2 = r2_score(y_test, pred_test)

print(f"{best_name}")
print(f"  Train RMSE: {train_rmse:.4f}")
print(f"  Test RMSE:  {test_rmse:.4f}")
print(f"  Test MAE:   {test_mae:.4f}")
print(f"  Test R2:    {test_r2:.4f}")
print(f"  Time:       {train_time:.1f}s")

## 9. Validation

In [ ]:
gap = test_rmse - train_rmse

checks = {
    f"Beats baseline ({baseline:.3f})": test_rmse < baseline,
    f"RMSE < {CONFIG['acceptable_rmse']}": test_rmse < CONFIG['acceptable_rmse'],
    f"Overfit gap < {CONFIG['max_overfit_gap']}": gap < CONFIG['max_overfit_gap'],
}

print("Validation:")
for check, passed in checks.items():
    print(f"  {check}: {'PASS' if passed else 'FAIL'}")

is_valid = all(checks.values())
print(f"\nReady for deployment: {'YES' if is_valid else 'NO'}")

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(11, 9))

axes[0,0].scatter(y_test, pred_test, alpha=0.2, s=3)
axes[0,0].plot([0,7], [0,7], 'r--')
axes[0,0].set_xlabel('Actual'); axes[0,0].set_ylabel('Predicted')
axes[0,0].set_title('Predictions')

res = y_test - pred_test
axes[0,1].hist(res, bins=50, edgecolor='black')
axes[0,1].axvline(0, color='red', linestyle='--')
axes[0,1].set_title(f'Residuals (mean={res.mean():.3f})')

axes[1,0].scatter(pred_test, res, alpha=0.2, s=3)
axes[1,0].axhline(0, color='red', linestyle='--')
axes[1,0].set_xlabel('Predicted'); axes[1,0].set_ylabel('Residual')

colors = ['gray', 'gray', 'green' if test_rmse < baseline else 'red']
axes[1,1].bar(['Mean', 'Naive', best_name], [mean_rmse, naive_rmse, test_rmse], color=colors)
axes[1,1].set_ylabel('RMSE')
axes[1,1].set_title('Comparison')

plt.tight_layout()
plt.show()

## 10. Error Analysis

In [ ]:
analysis = pd.DataFrame({'actual': y_test, 'pred': pred_test})
analysis['bucket'] = pd.cut(analysis['actual'], bins=[-1, 1, 3, 5, 7], labels=['0-1', '2-3', '4-5', '6-7'])

print("RMSE by segment:")
for bucket, g in analysis.groupby('bucket'):
    rmse = np.sqrt(((g['actual'] - g['pred'])**2).mean())
    print(f"  {bucket} days: {rmse:.3f} (n={len(g):,})")

## 11. Feature Importance

In [ ]:
imp = pd.DataFrame({
    'feature': features, 
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print(imp.to_string(index=False))

In [ ]:
sample = X_train.sample(n=min(1000, len(X_train)), random_state=SEED)
explainer = shap.TreeExplainer(model)
shap_vals = explainer.shap_values(sample)
mean_shap = np.abs(shap_vals).mean(axis=0)

shap.summary_plot(shap_vals, sample, feature_names=features, show=False, max_display=15)
plt.tight_layout()
plt.show()

## 12. Feature Selection

In [ ]:
report = pd.DataFrame({'feature': features, 'imp': model.feature_importances_, 'shap': mean_shap})
thresh = CONFIG['importance_threshold']
keep = report[(report['imp'] > thresh) | (report['shap'] > thresh)]['feature'].tolist()

shap_map = dict(zip(report['feature'], report['shap']))
corr_mat = X_train[keep].corr().abs()
drop = set()
for i, j in zip(*np.where(np.triu(corr_mat.values, k=1) > CONFIG['correlation_threshold'])):
    a, b = corr_mat.columns[i], corr_mat.columns[j]
    drop.add(b if shap_map.get(a, 0) >= shap_map.get(b, 0) else a)

final_features = [f for f in keep if f not in drop]
print(f"Features: {len(features)} -> {len(keep)} -> {len(final_features)}")

In [ ]:
model_final = ModelClass(**params)
model_final.fit(X_train[final_features], y_train)

pred_final = model_final.predict(X_test[final_features])
final_rmse = np.sqrt(mean_squared_error(y_test, pred_final))
final_mae = mean_absolute_error(y_test, pred_final)
final_r2 = r2_score(y_test, pred_final)
final_gap = final_rmse - np.sqrt(mean_squared_error(y_train, model_final.predict(X_train[final_features])))

print(f"Final model ({len(final_features)} features):")
print(f"  RMSE: {final_rmse:.4f} (delta: {final_rmse - test_rmse:+.4f})")
print(f"  MAE:  {final_mae:.4f}")
print(f"  R2:   {final_r2:.4f}")

final_valid = (final_rmse < baseline) and (final_rmse < CONFIG['acceptable_rmse']) and (final_gap < CONFIG['max_overfit_gap'])
print(f"\nValid: {'YES' if final_valid else 'NO'}")

## 12b. Production Distribution Check

In [ ]:
test_dates = dates_test.unique()
test_min = pd.Timestamp(test_dates.min()).strftime('%Y-%m-%d')
test_max = pd.Timestamp(test_dates.max()).strftime('%Y-%m-%d')
natural_sample_size = 200000

query_natural = f"""
SELECT * FROM (
    SELECT * FROM {CONFIG['table']}
    WHERE date_utc >= '{test_min}'
      AND date_utc <= '{test_max}'
) SAMPLE ({natural_sample_size} ROWS)
"""

spine_natural = session.sql(query_natural)
df_natural = fs.generate_training_set(
    spine_df=spine_natural, 
    features=feature_views, 
    spine_label_cols=["ACTIVE_DAYS_IN_WEEK"]
).to_pandas().fillna(0)

X_natural = df_natural[final_features]
y_natural = df_natural['ACTIVE_DAYS_IN_WEEK']

pred_natural = model_final.predict(X_natural)
natural_rmse = np.sqrt(mean_squared_error(y_natural, pred_natural))
natural_mae = mean_absolute_error(y_natural, pred_natural)

print("Production Distribution Evaluation (sampled):")
print(f"  Rows: {len(df_natural):,}")
print(f"  RMSE: {natural_rmse:.4f}")
print(f"  MAE:  {natural_mae:.4f}")
print(f"")
print("Target distribution (natural):")
print(y_natural.value_counts(normalize=True).sort_index().apply(lambda x: f"{x:.1%}").to_string())
print(f"")
print(f"Balanced test RMSE: {final_rmse:.4f}")
print(f"Natural test RMSE:  {natural_rmse:.4f}")
print(f"Delta: {natural_rmse - final_rmse:+.4f}")

## 13. Save

In [ ]:
run_name = f"run_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
with exp.start_run(run_name=run_name):
    exp.log_metric("rmse", final_rmse)
    exp.log_metric("mae", final_mae)
    exp.log_metric("training_time_seconds", train_time)
    
    exp.log_param("model_type", best_name)
    exp.log_param("baseline_rmse", float(baseline))
    exp.log_param("cv_rmse", float(study.best_value))
    exp.log_param("overfit_gap", float(final_gap))
    exp.log_param("n_features", len(final_features))
    exp.log_param("n_train_samples", len(X_train))
    exp.log_param("n_test_samples", len(X_test))
    exp.log_param("lookback_days", CONFIG['lookback_days'])
    exp.log_param("features", json.dumps(final_features))
    exp.log_param("config", json.dumps(CONFIG))
    
    exp.log_model(
        model=model_final,
        model_name="user_activity_model",
        metrics={"rmse": final_rmse, "mae": final_mae, "training_time_seconds": train_time},
        sample_input_data=X_train[final_features].head()
    )

print("Saved to registry")
print(f"\nFeatures: {final_features}")

In [ ]:
print("="*50)
print("SUMMARY")
print("="*50)
print(f"Data:     {len(df):,} rows, {df['DATE_UTC'].nunique()} days")
print(f"Split:    {len(X_train):,} / {len(X_test):,}")
print(f"Features: {len(features)} -> {len(final_features)}")
print(f"")
print(f"Baseline: {baseline:.4f}")
print(f"Model:    {final_rmse:.4f} ({(baseline - final_rmse)/baseline*100:.1f}% better)")
print(f"")
print(f"Deploy:   {'YES' if final_valid else 'NO'}")
print("="*50)